# Ensambles


### Imports

In [517]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

### Funciones

In [518]:
def dia_semana(numero):
    dias = {
        0: 'Lunes',
        1: 'Martes',
        2: 'Miercoles',
        3: 'Jueves',
        4: 'Viernes',
        5: 'Sabado',
        6: 'Domingo'
    }
    
    return dias.get(numero)

In [519]:
def normalizar_descripciones(descripcion):
    
    descripcion = descripcion.replace(',', ' ')
    descripcion = descripcion.replace('.', ' ')
    descripcion = descripcion.replace('<', ' ')
    descripcion = descripcion.replace('>', ' ')
    descripcion = descripcion.replace('\n', ' ')
    descripcion = descripcion.replace('\strong', ' ')
    descripcion = descripcion.replace('\bold', ' ')
    descripcion = descripcion.lower()
    
    return descripcion

def contar_palabras(titulo, palabras):
    
    contadas = 0
    spliteado = titulo.split()
    
    for palabra in palabras:
        contadas += spliteado.count(palabra)
        
    return contadas

def contar_frases(columna, frases):
    
    contadas = 0
    
    for frase in frases:
        
        contadas += columna.count(frase)
        
    return contadas

### Reads
train.csv -> train
<br>
test.csv -> test

In [520]:
train = pd.read_csv('./train.csv')
#test = pd.read_csv('../data/test.csv')

### Preparo el modelo 

**Feature Direción:**

In [560]:
df = train

In [561]:
DIRINVALIDAS = set(['Calle','domicilio conocido','calle','CALLE','--','Sin nombre ','1','.','-','x #x','0','...','S/N'])


df.loc[df.direccion.isin(DIRINVALIDAS), 'direccion'] = 0
df.loc[df['direccion'] != 0, 'direccion'] = 1

df['titulo'] = df['titulo'].fillna(value = ".")
df['titulo'] = df['titulo'].apply(normalizar_descripciones)
df['descripcion'] = df['descripcion'].fillna(value = ".")
df['descripcion'] = df['descripcion'].apply(normalizar_descripciones)

df['fecha'] = pd.to_datetime(df['fecha'])

**Feature Titulo y Descripcion:**

In [562]:
    df['calefaccion'] = df['titulo'].apply(contar_palabras, palabras=["calefaccion", "calefacción", "calefaccionado", "aireacondicinado",
                        "acondicionado", "estufa","chimenea","calido","templado"]) +df['descripcion'].apply(contar_palabras, palabras=["calefaccion", 
                    "calefacción", "calefaccionado", "aireacondicinado", "acondicionado", "estufa","chimenea","calido","templado"])

    df['suite'] = df['titulo'].apply(contar_palabras, palabras=["suite"])+df['descripcion'].apply(contar_palabras, palabras=["suite"])

    df['avenida'] = df['titulo'].apply(contar_palabras, palabras=["avenida", "av", "avenidas", "bulevar", 
               "boulevard", "paseo", "vía"])+df['descripcion'].apply(contar_palabras, palabras=["avenida", "av", 
                "avenidas", "bulevar","boulevard", "paseo", "vía"])

    df['gim'] = df['titulo'].apply(contar_palabras, palabras=["gimnasio", "gimnásio", "entrenamiento", "gim", "gym", "fit",
            "ejercicio", "gimnasia","atletismo", "cancha"])+df['descripcion'].apply(contar_palabras, palabras=["gimnasio",
            "gimnásio", "entrenamiento", "gim", "gym", "fit","ejercicio", "gimnasia","atletismo", "cancha"])

    #df['cochera'] = df['titulo'].apply(contar_palabras, palabras=["cochera", "cocheras", "garage", 
    #                "garages", "garaje", "garajes"])+df['descripcion'].apply(contar_palabras, palabras=["cochera", 
    #                "cocheras", "garage","garages", "garaje", "garajes"])

    df['ubicacion'] = df['titulo'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central","esquina"])+df['descripcion'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                "centrica", "céntrico", "céntrica","central","esquina"])

    df['balcon'] = df['titulo'].apply(contar_palabras, palabras=["balcon", "balcón", "terraza", "palco", "mirador", 
            "balconcillo","azotea", "solana"])+df['descripcion'].apply(contar_palabras, palabras=["balcon", "balcón", 
            "terraza", "palco", "mirador", "balconcillo","azotea", "solana"])

    df['camaraseg'] = df['titulo'].apply(contar_palabras, palabras=["camara", "cámara", "cámaras", "camaras","seguridad",
                "guardia","seguro","protegido"]) + df['descripcion'].apply(contar_palabras, palabras=["camara", "cámara",
                "cámaras", "camaras","seguridad","guardia","seguro","protegido"])

    df['parque'] = df['titulo'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", "glorieta","jardin",
            "jardín","patio"])+df['descripcion'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", 
            "glorieta","jardin","jardín","patio"])

    df['amoblado'] = df['titulo'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble","decorado",
                "listo"]) + df['descripcion'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble",
                "decorado","listo"])

    df['bañera'] = df['titulo'].apply(contar_palabras, palabras=["bañera", "hidromasaje", "hidro", "tina",
            "jacuzzi","jacuzi","yacuzi"]) + df['descripcion'].apply(contar_palabras, palabras=["bañera", "hidromasaje", 
            "hidro", "tina","jacuzzi","jacuzi","yacuzi"])


    #df['estreno'] = df['titulo'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
    #            "innovador"]) + df['descripcion'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
    #            "iuevonovador"])

    df['transporte'] = df['titulo'].apply(contar_palabras, palabras=["subte", "subterraneo", "subterráneo","metro", 
                "estacion", "estación", "tren","subestacion", "subestación", "ferrocarril","metrobús", "metrobus", 
                "trolebus","trolebús", "bus", "bús"]) + df['descripcion'].apply(contar_palabras, palabras=["subte", 
                "subterraneo", "subterráneo","metro", "estacion", "estación", "tren","subestacion", "subestación", 
                "ferrocarril","metrobús", "metrobus", "trolebus","trolebús", "bus", "bús"])

    df['pileta'] = df['titulo'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])+ df['descripcion'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])
                                       
    df['lujo'] = df['titulo'].apply(contar_palabras, palabras=["lujo", "delujo", "deluxe", "delúxe", "lujosa", "lujoso", 
            "lujosas", "lujosos","exclusivo","vip"]) + df['descripcion'].apply(contar_palabras, palabras=["lujo", 
            "delujo","deluxe", "delúxe", "lujosa", "lujoso", "lujosas", "lujosos","exclusivo","vip"])

    df['humilde'] = df['titulo'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica","credito","crédito","oferta","oferton",
            "imperdible"]) + df['descripcion'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
            "económico", "económica", "barata", "barato", "accesible", "baratillo",
            "baratilla", "rebajado", "ganga", "asequible", "módico", "módica",
            "credito","crédito","oferta","oferton","imperdible"]) 

    df['ventana'] = df['titulo'].apply(contar_palabras, palabras=["ventana", "ventanas", 
            "vista", "ventanal","vistas","cristal"]) + df['descripcion'].apply(contar_palabras, palabras=["ventana",
            "ventanas", "vista", "ventanal","vistas","cristal"])

    #df['nuevo'] = df['titulo'].apply(contar_palabras, palabras=["reciente", "recien", "recién", "nueva", "nuevo", "nuevas", 
    #            "nuevos", "estrenar"]) + df['descripcion'].apply(contar_palabras, palabras=["reciente", "recien", "recién", 
    #            "nueva", "nuevo", "nuevas", "nuevos", "estrenar"])

    df['luz'] = df['titulo'].apply(contar_palabras, palabras=["luz", "luminoso", "luminosa","claridad", "luminiscencia", 
        "luminosidad", "iluminación","iluminacion"]) + df['descripcion'].apply(contar_palabras, palabras=["luz", 
        "luminoso", "luminosa","claridad", "luminiscencia","luminosidad", "iluminación","iluminacion"])

    #df['bueno'] = df['titulo'].apply(contar_palabras, palabras=["bueno", "buena", "buenas", "buenos","excelente", 
    #            "excelentes","increible","espectacular"]) + df['descripcion'].apply(contar_palabras, palabras=["bueno", 
    #            "buena", "buenas", "buenos","excelente", "excelentes","increible","espectacular"])

    df['contable'] = df['titulo'].apply(contar_palabras, palabras=["precio"]) + df['descripcion'].apply(contar_palabras, palabras=["precio"])

    #Nuevos Feat

    df['agente'] = df['descripcion'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"]) \
             + df['titulo'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"])

    df['garante'] = df['descripcion'].apply(contar_palabras, palabras=["garante", "garantía", "fiador", "garantizador", "avalista", "garantia",
                "defensor", "garantías", "garantes", "codeudor"]) + df['titulo'].apply(contar_palabras, palabras=["garante", 
                "garantía", "fiador", "garantizador", "avalista", "garantia", "defensor", "garantías", "garantes", "codeudor"])

    df['finanza'] =  df['descripcion'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario"\
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocréditp", "ahorro"]) + \
                 df['titulo'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario" \
                 ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocrédito", "ahorro"]) 

    df['turismo'] =  df['descripcion'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])\
                 + df['titulo'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                   "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])

    df['longitud_desc'] =  df['descripcion'].apply(lambda x: len(x.split()))

    #Nuevas de V4

    df['longitud_titulo'] = df['titulo'].apply(lambda x: len(x))

    df['cant_!'] = df['titulo'].apply(lambda x: x.count('!')) 

    df['tranquilidad'] = df['descripcion'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])\
                 + df['titulo'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                        "paz", "calma", "calmo", "quietud"])

    df['reparacion'] = df['descripcion'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada",
                                                                     "refaccion","refacción","refacciones"])\
                 + df['titulo'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                            "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada"
                                                                     "refaccion","refacción","refacciones"])

    df['mascotas'] = df['descripcion'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])\
                 + df['titulo'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                "gatos", "gato", "gata", "animal", "animales"])

    df['accesibilidad'] = df['descripcion'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])\
                 + df['titulo'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])

    df['normas'] =  df['descripcion'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])\
                 + df['titulo'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                "ilegal", "legal", "multa", "infraccion", "infracción"])

    df['beneficios'] = df['descripcion'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])\
                 + df['titulo'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                "agregado", "gratuito", "gratuitamente", \
                                                "regalo"])

    df['conexion'] = df['descripcion'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])\
                 + df['titulo'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                   "señal"])

    df['servicios_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["servicio", "servicios"])

    df['metros_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["metros","m2"])

    df['acabados'] = df['descripcion'].apply(contar_palabras, palabras = ["acabados","acabado","terminacion","terminación"])

    df['plusvalia'] = df['descripcion'].apply(contar_palabras, palabras = ["plusvalia", "plusvalía"])

    #Nuevos de V5

    df['cocina'] = df['descripcion'].apply(contar_palabras, palabras=['cocina', 'cocinas'])+df['titulo'].apply(contar_palabras, palabras=['cocina', 'cocinas'])

    df['alberca'] = df['descripcion'].apply(contar_palabras, palabras=['alberca'])+df['titulo'].apply(contar_palabras, palabras=['alberca'])

    df['negacion'] = df['descripcion'].apply(contar_palabras, palabras = ["no"])+df['titulo'].apply(contar_palabras, palabras = ["no"])

    df['variospisos'] = df['descripcion'].apply(contar_palabras, palabras = ["escalera","escaleras",
                    "ascensor", "elevador", "escalinata", "gradas", "escalerilla"])+ df['titulo'].apply(contar_palabras, palabras = ["escalera","escaleras",
                    "ascensor", "elevador", "escalinata", "gradas", "escalerilla"])

    df['vestidor'] = df['descripcion'].apply(contar_palabras, palabras = ["vestidor"])+df['titulo'].apply(contar_palabras, palabras = ["vestidor"])

    #Nuevos de V7

    df['popularidad'] = df['descripcion'].apply(contar_palabras, palabras=["popular", "famoso", "famosa", "famosos", "populares", \
                                                        "conocido", "conocida", "conocidos"])\
                 + df['titulo'].apply(contar_palabras, palabras=["popular", "famoso", "famosa", "famosos", "populares", \
                                                        "conocido", "conocida", "conocidos"])

    df['aeropuerto_cercano'] = df['descripcion'].apply(contar_palabras, palabras=["aeropuerto", "aeropuertos", \
                                                                             "aeródromo", "aerodromo", "estacion"\
                                                                             "estación"])\
                 + df['titulo'].apply(contar_palabras, palabras=["aeropuerto", "aeropuertos", \
                                                                             "aeródromo", "aerodromo", "estacion"\
                                                                             "estación"])

    df['traslado'] = df['descripcion'].apply(contar_palabras, palabras=["traslado", "trasladar", "llevamos", \
                                                                   "llevar", "traslada"])\
                 + df['titulo'].apply(contar_palabras, palabras=["traslado", "trasladar", "llevamos", \
                                                                   "llevar", "traslada"])

    #df['extranjero'] = df['descripcion'].apply(contar_palabras, palabras=["extranjero", "extranjera", "extranjeros"])\
    #                + df['titulo'].apply(contar_palabras, palabras=["extranjero", "extranjera", "extranjeros"])

    df['dueño'] = df['descripcion'].apply(contar_palabras, palabras=["dueño", "dueña", "dueños", "propietario", "propietarios"])\
                 + df['titulo'].apply(contar_palabras, palabras=["dueño", "dueña", "dueños", "propietario", "propietarios"])

    df['monoambiente'] = df['descripcion'].apply(contar_palabras, palabras=["monoambiente", "monoambientes"]) \
                     + df['descripcion'].apply(contar_frases, frases=["un ambiente", "unico ambiente", "único ambiente"]) \
                     + df['titulo'].apply(contar_palabras, palabras=["monoambiente", "monoambientes"]) \
                     + df['titulo'].apply(contar_frases, frases=["un ambiente", "unico ambiente", "único ambiente"])

    #La renta fija es un tipo de inversión formada por todos los activos financieros en los que el emisor está 
    #obligado a realizar pagos en una cantidad y en un período de tiempo previamente establecidos
    #Es decir, en la renta fija el emisor garantiza la devolución del capital invertido y una cierta rentabilidad.

    df['renta'] =  df['descripcion'].apply(contar_palabras, palabras=["renta", "devolucion", "devolución", \
                                                                 "devolver", "devolvemos", "devuelto", "depósito", \
                                                                 "deposito", "regreso"]) \
              + df['descripcion'].apply(contar_frases, frases=["renta fija", "regreso del depósito", "regreso del deposito"]) \
              + df['titulo'].apply(contar_palabras, palabras=["renta", "devolucion", "devolución", \
                                                                 "devolver", "devolvemos", "devuelto", "depósito", \
                                                                 "deposito", "regreso"]) \
              + df['titulo'].apply(contar_frases, frases=["renta fija", "regreso del depósito", "regreso del deposito"])

    df['oportunidad'] = df['descripcion'].apply(contar_palabras, palabras=["opotunidad", "chance", "imperdible", \
                                                                      "aprovecha", "aprovechar"]) \
              + df['descripcion'].apply(contar_frases, frases=["ultima oportunidad", "última oportunidad", \
                                                              "no te lo pierdas", "no lo pierdas", "no dejes pasar"]) \
              + df['titulo'].apply(contar_palabras, palabras=["opotunidad", "chance", "imperdible", \
                                                                      "aprovecha", "aprovechar"]) \
              + df['titulo'].apply(contar_frases, frases=["ultima oportunidad", "última oportunidad", \
                                                              "no te lo pierdas", "no lo pierdas", "no dejes pasar"]) 

    df['antiguedad_prop'] = df['descripcion'].apply(contar_palabras, palabras=["antiguo", "antigua", "antigüedad", \
                                                                          "antiguedad", "viejo", "vieja", \
                                                                          "clásico", "clasico", "clásica", "clasica"]) \
              + df['titulo'].apply(contar_palabras, palabras=["antiguo", "antigua", "antigüedad", \
                                                                          "antiguedad", "viejo", "vieja", \
                                                                          "clásico", "clasico", "clásica", "clasica"]) 

    df['niños'] = df['descripcion'].apply(contar_palabras, palabras=["niños", "niñas", "niño", "niña", "bebé", \
                                                                  "bebes", "bebe", "hijo", "hija", "hijos", "hijas"]) \
              + df['titulo'].apply(contar_palabras, palabras=["niños", "niñas", "niño", "niña", "bebé", \
                                                                  "bebes", "bebe", "hijo", "hija", "hijos", "hijas"]) 

    df['fotos'] = df['descripcion'].apply(contar_palabras, palabras=["foto", "fotos", "imagenes", "imágenes", "imagen", \
                                                                "ilustrado", "ilustración", "ilustracion", "fotografia", \
                                                                "fotografía", "fotografías", "fotografias"]) \
              + df['titulo'].apply(contar_palabras, palabras=["foto", "fotos", "imagenes", "imágenes", "imagen", \
                                                                "ilustrado", "ilustración", "ilustracion", "fotografia", \
                                                                "fotografía", "fotografías", "fotografias"]) 

    df['lavadero'] = df['descripcion'].apply(contar_palabras, palabras=["lavadero", "lavadora", "lavarropa", "ropa"]) \
                + df['descripcion'].apply(contar_frases, frases=["cuarto de lavado"]) \
                + df['titulo'].apply(contar_frases, frases=["cuarto de lavado"]) \
              + df['titulo'].apply(contar_palabras, palabras=["lavadero", "lavadora", "lavarropa", "ropa"]) 

    #df['camas_separadas'] =  df['descripcion'].apply(contar_frases, frases=["camas separadas", "camas individuales"]) \
    #              + df['titulo'].apply(contar_frases, frases=["camas separadas", "camas individuales"])

    df['$'] = df['descripcion'].apply(contar_palabras, palabras=["$","pesos"]) \
              + df['titulo'].apply(contar_palabras, palabras=["$","pesos"])

    df['trabajo'] = df['descripcion'].apply(contar_palabras, palabras=["oficina", "oficio", "trabajo", "trabajador"]) \
              + df['titulo'].apply(contar_palabras, palabras=["oficina", "oficio", "trabajo", "trabajador"])

    #df['gas'] = df['descripcion'].apply(contar_palabras, palabras=["gas"]) \
    #              + df['titulo'].apply(contar_palabras, palabras=["gas"])

    df['agua'] = df['descripcion'].apply(contar_palabras, palabras=["cañeria", "cañería", "cañerías", "cañerias", \
                                                               "agua", "aguas", "tubería", "tuberia", "tuberías", "tuberias"]) \
              + df['titulo'].apply(contar_palabras, palabras=["cañeria", "cañería", "cañerías", "cañerias", \
                                                               "agua", "aguas", "tubería", "tuberia", "tuberías", "tuberias"])

    df['amenidad'] = df['descripcion'].apply(contar_palabras, palabras=["ameno", "amenidad", "amenidades"]) \
              + df['titulo'].apply(contar_palabras, palabras=["ameno", "amenidad", "amenidades"])

    df['broker'] = df['descripcion'].apply(contar_palabras, palabras=["broker", "brokers"]) + \
               df['titulo'].apply(contar_palabras, palabras=["broker", "brokers"])

**Feature Metros:**

In [563]:
df['metrosdescubiertos'] = abs(df['metrostotales']-df['metroscubiertos'])
df['relacionmetros'] = df['metrostotales']/df['metroscubiertos']

In [564]:
def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.35,random_state=42)
    return x_train, x_test, y_train, y_test

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

def minmax(serie):
    
    return (serie-serie.min())/(serie.max()-serie.min())

def mean_target_encoding(train, nombrefeat, nombretarget):
    
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean)
    train[nombrefeat + "_mean"]=(temp-temp.min())/(temp.max()-temp.min())
    
    return train

def mean_target_decoding(x_test, nombrefeat, x_train):
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 

    return x_test

def armar_set_2(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo','direccion', 'id'], axis = 1)
    
    #Adiciono servicios
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    #Dropeo los de bajo scoring
    train = train.drop(['escuelascercanas', 'usosmultiples',"centroscomercialescercanos"], axis = 1)
    
    train['fecha'] = pd.to_datetime(train['fecha']).dt.year
    
    #"Normalizo" la antiguedad
    #train['antiguedad'] = minmax(train['antiguedad'])
    
    #Elimino los residuos
    #train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                       # 'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train
    x_test["precio"] = y_test
    
    
    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']

    PMM = df1.groupby('idzona').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')


    df1 = x_train.dropna(subset=['metroscubiertos'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    
    df1 = x_train.dropna(subset=['metrosdescubiertos'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMean"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')
    
    
    #x = x_test.fillna(0)
    #x_test['primera_pred'] = (x['precioMetroCubMean']*x['metroscubiertos']+x['precioMetroMean']*x['metrostotales'])/2
    
    #x = x_train.fillna(0)
    #x_train['primera_pred'] = (x['precioMetroCubMean']*x['metroscubiertos']+x['precioMetroMean']*x['metrostotales'])/2

    df1 = x_train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']
    
    PMM = df1.groupby('ciudad').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metroscubiertos'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    df1 = x_train.dropna(subset=['metrosdescubiertos'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMeanC"})

    x_train = x_train.merge(PMM, on='ciudad',how='left')
    x_test = x_test.merge(PMM, on='ciudad',how='left')
    
    
    
    
    PMM = x_train.groupby('idzona').agg({'precio': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"precio": "precioMeanZona"})

    x_train = x_train.merge(PMM, on='idzona',how='left')
    x_test = x_test.merge(PMM, on='idzona',how='left')

    
    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "provincia", "precio") 
    x_train_mean['precio'] = y_train
    #x_train_mean = mean_target_encoding(x_train_mean, "tipodepropiedad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    x_train = mean_target_encoding(x_train_mean, "idzona", "precio")

    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "provincia", x_train_mean)
    #x_test = mean_target_decoding(x_test, "tipodepropiedad", x_train_mean)
    x_test = mean_target_decoding(x_test, "ciudad", x_train_mean)
    x_test = mean_target_decoding(x_test, "idzona", x_train_mean)
    
    backup = x_train_mean
    
    x_train = x_train_mean.drop(["precio","provincia","tipodepropiedad","ciudad"], axis=1)
    x_test.drop(["precio","provincia","tipodepropiedad", "ciudad"], axis=1, inplace = True)
    
    #Voy a minmaxmizar los metros (todos)
    x_train['metroscubiertos2'] = minmax(x_train['metroscubiertos'])
    x_train['metrostotales2'] = minmax(x_train['metrostotales'])
    x_train['metrosdescubiertos2'] = minmax(x_train['metrosdescubiertos'])
    #x_train['lat'] = minmax(x_train['lat'])
    #x_train['lng'] = minmax(x_train['lng'])
    
    x_test['metroscubiertos2'] = minmax(x_test['metroscubiertos'])
    x_test['metrostotales2'] = minmax(x_test['metrostotales'])
    x_test['metrosdescubiertos2'] = minmax(x_test['metrosdescubiertos'])
    #x_test['lat'] = minmax(x_test['lat'])
    #x_test['lng'] = minmax(x_test['lng'])
    
    from sklearn.impute import SimpleImputer


    return x_train, x_test, y_train, y_test, backup

In [565]:
x_train, x_test, y_train, y_test, backup = armar_set_2(df)



# divido el df 

/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [566]:
   
    y = y_train.values

    x = x_train.values

    Y_test = y_test.values

    X_test = x_test.values

### Creo LightGBM data containers

In [567]:
categorical_features = [c for c, col in enumerate(train.columns) if 'cat' in col]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(X_test, label=Y_test)

In [568]:
#Lo entreno

model = lightgbm.LGBMRegressor(colsample_bytree=0.9354186748964698,learning_rate= 0.05,
                                min_child_samples=80,min_child_weight = 0.01,
                                num_leaves=133, reg_alpha=1e-05,reg_lambda=10,subsample=0.7064877172122896,
                                max_depth=-1,max_bin=828,metric = 'mae',n_estimators= 1000,
                                random_state=314)
model.fit(x,y)

LGBMRegressor(boosting_type='gbdt', class_weight=None,
              colsample_bytree=0.9354186748964698, importance_type='split',
              learning_rate=0.05, max_bin=828, max_depth=-1, metric='mae',
              min_child_samples=80, min_child_weight=0.01, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=133, objective=None,
              random_state=314, reg_alpha=1e-05, reg_lambda=10, silent=True,
              subsample=0.7064877172122896, subsample_for_bin=200000,
              subsample_freq=0)

In [569]:
prediccionLGBM = model.predict(X_test)

In [571]:
print(mean_absolute_error(y_test, prediccionLGBM))

489888.5090115646


In [75]:
model.feature_importances_

array([ 5753,  2781,  3111,  2557, 13874,  9790,  5660,  4443,  4244,
        2529,   154,   169,   162,    28,   190,   305,   762,   530,
         169,   861,    71,   245,    75,    93,   351,   119,   412,
         157,   283,    89,    93,   214,   320,  5464,  4681,    82,
          44,   363,    32,    21,    39,    64,    31,   671,   649,
         306,    55,   664,   286,   477,   158,   469,    11,    19,
          36,    12,    15,   124,    17,   154,    37,    63,   112,
         164,   149,   109,    91,     0,  2791,  3782,  3311,   310,
        9222, 11075,   786,  4001,  3564,   107,  2240,  1738,  2152,
        3289,  5307,  1257,   561,   244], dtype=int32)

# xgb

In [131]:
xgb_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, eval_metric='mae',
             gamma=0.015, importance_type='gain', learning_rate=0.025,
             max_delta_step=0, max_depth=11, min_child_weight=1, missing=None,
             n_estimators=3500, n_jobs=None, nthread=-1, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.733, verbosity=1)


#Fitteo
xgb_model.fit(x_train, y_train)


/home/belu/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[23:17:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, eval_metric='mae',
             gamma=0.015, importance_type='gain', learning_rate=0.025,
             max_delta_step=0, max_depth=11, min_child_weight=1, missing=None,
             n_estimators=3500, n_jobs=None, nthread=-1, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.733, verbosity=1)

In [132]:
prediccion = xgb_model.predict(x_test)

In [133]:
#Estimo MAE
print(mean_absolute_error(y_test, prediccion))

474806.72199925594


# Ensamble

In [148]:
ensamble = (prediccion*0.8 + prediccionLGBM*0.19)

In [149]:
#Estimo MAE
print(mean_absolute_error(y_test, ensamble))

472634.3849178469


In [166]:
x_train.head()

,direccion,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,...,precioMetroMean,precioMetroCubMean,precioMetroDesMean,precioMetroMeanC,precioMetroCubMeanC,precioMetroDesMeanC,provincia_mean,tipodepropiedad_mean,ciudad_mean,idzona_mean
0,1,5.0,6.0,0.0,2.0,0.790094,0.384434,NaN,NaN,NaN,...,NaN,NaN,NaN,28695.222989,27415.794873,inf,1.000000,0.107951,0.211064,NaN
1,1,0.0,3.0,NaN,3.0,0.450472,0.450472,23144.0,19.356202,-99.19943,...,30767.337814,27986.622220,inf,27229.640691,25112.544940,inf,1.000000,0.097282,0.205012,0.189177
2,1,25.0,2.0,1.0,1.0,0.096698,NaN,24896.0,NaN,NaN,...,17195.972169,16542.209508,inf,20213.100946,18602.528363,inf,1.000000,0.097282,0.207949,0.134824
3,1,0.0,3.0,1.0,2.0,0.250000,NaN,110512.0,NaN,NaN,...,10318.099099,9226.096999,5.098259e+04,12719.863365,12124.136286,inf,0.413236,0.097317,0.200124,0.248493
4,1,10.0,3.0,3.0,2.0,0.271226,0.318396,55790.0,NaN,NaN,...,15973.481250,14437.282876,inf,18558.600118,16848.802187,inf,0.745954,0.097317,0.202751,0.205870


In [161]:
x_train['precioMetroDesMeanC'].value_counts()

inf             153478
1.125264e+04       150
2.880516e+04        87
3.573994e+04        76
4.131262e+04        50
                 ...  
3.500000e+04         1
1.333333e+05         1
4.758621e+04         1
2.884615e+04         1
4.418605e+04         1
Name: precioMetroDesMeanC, Length: 285, dtype: int64

# Prediccion

In [184]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [185]:
def preparar_df(df):
    DIRINVALIDAS = set(['Calle','domicilio conocido','calle','CALLE','--','Sin nombre ','1','.','-','x #x','0','...','S/N'])


    df.loc[df.direccion.isin(DIRINVALIDAS), 'direccion'] = 0
    df.loc[df['direccion'] != 0, 'direccion'] = 1

    df['titulo'] = df['titulo'].fillna(value = ".")
    df['titulo'] = df['titulo'].apply(normalizar_descripciones)
    df['descripcion'] = df['descripcion'].fillna(value = ".")
    df['descripcion'] = df['descripcion'].apply(normalizar_descripciones)

    df['fecha'] = pd.to_datetime(df['fecha'])


    df['calefaccion'] = df['titulo'].apply(contar_palabras, palabras=["calefaccion", "calefacción", "calefaccionado", "aireacondicinado",
                        "acondicionado", "estufa","chimenea"]) +df['descripcion'].apply(contar_palabras, palabras=["calefaccion", 
                        "calefacción", "calefaccionado", "aireacondicinado", "acondicionado", "estufa","chimenea"])

    df['suite'] = df['titulo'].apply(contar_palabras, palabras=["suite"])+df['descripcion'].apply(contar_palabras, palabras=["suite"])

    df['avenida'] = df['titulo'].apply(contar_palabras, palabras=["avenida", "av", "avenidas", "bulevar", 
                   "boulevard", "paseo", "vía"])+df['descripcion'].apply(contar_palabras, palabras=["avenida", "av", 
                    "avenidas", "bulevar","boulevard", "paseo", "vía"])

    df['gim'] = df['titulo'].apply(contar_palabras, palabras=["gimnasio", "gimnásio", "entrenamiento", "gim", "gym", "fit",
                "ejercicio", "gimnasia","atletismo", "cancha"])+df['descripcion'].apply(contar_palabras, palabras=["gimnasio",
                "gimnásio", "entrenamiento", "gim", "gym", "fit","ejercicio", "gimnasia","atletismo", "cancha"])

    #df['cochera'] = df['titulo'].apply(contar_palabras, palabras=["cochera", "cocheras", "garage", 
    #                "garages", "garaje", "garajes"])+df['descripcion'].apply(contar_palabras, palabras=["cochera", 
    #                "cocheras", "garage","garages", "garaje", "garajes"])

    df['ubicacion'] = df['titulo'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                    "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                    "centrica", "céntrico", "céntrica","central"])+df['descripcion'].apply(contar_palabras, palabras=["ubicacion", "ubicación", "locacion", 
                    "locación", "localizacion", "localización","ubicado", "ubicada", "centro", "centrico",
                    "centrica", "céntrico", "céntrica","central"])

    df['balcon'] = df['titulo'].apply(contar_palabras, palabras=["balcon", "balcón", "terraza", "palco", "mirador", 
                "balconcillo","azotea", "solana"])+df['descripcion'].apply(contar_palabras, palabras=["balcon", "balcón", 
                "terraza", "palco", "mirador", "balconcillo","azotea", "solana"])

    df['camaraseg'] = df['titulo'].apply(contar_palabras, palabras=["camara", "cámara", "cámaras", "camaras","seguridad",
                    "guardia","seguro","protegido"]) + df['descripcion'].apply(contar_palabras, palabras=["camara", "cámara",
                    "cámaras", "camaras","seguridad","guardia","seguro","protegido"])

    df['parque'] = df['titulo'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", "glorieta","jardin",
                "jardín","patio"])+df['descripcion'].apply(contar_palabras, palabras=["parque", "plaza", "plazoleta", 
                "glorieta","jardin","jardín","patio"])

    df['amoblado'] = df['titulo'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble","decorado",
                    "listo"]) + df['descripcion'].apply(contar_palabras, palabras=["muebles", "amoblado", "mueble",
                    "decorado","listo"])

    df['bañera'] = df['titulo'].apply(contar_palabras, palabras=["bañera", "hidromasaje", "hidro", "tina",
                "jacuzzi","jacuzi","yacuzi"]) + df['descripcion'].apply(contar_palabras, palabras=["bañera", "hidromasaje", 
                "hidro", "tina","jacuzzi","jacuzi","yacuzi"])


    #df['estreno'] = df['titulo'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
    #            "innovador"]) + df['descripcion'].apply(contar_palabras, palabras=["nuevo", "nueva", "estrenar","estreno",
    #            "iuevonovador"])

    df['transporte'] = df['titulo'].apply(contar_palabras, palabras=["subte", "subterraneo", "subterráneo","metro", 
                    "estacion", "estación", "tren","subestacion", "subestación", "ferrocarril","metrobús", "metrobus", 
                    "trolebus","trolebús", "bus", "bús"]) + df['descripcion'].apply(contar_palabras, palabras=["subte", 
                    "subterraneo", "subterráneo","metro", "estacion", "estación", "tren","subestacion", "subestación", 
                    "ferrocarril","metrobús", "metrobus", "trolebus","trolebús", "bus", "bús"])

    df['pileta'] = df['titulo'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])+ df['descripcion'].apply(contar_palabras, palabras=["piscina", "pileta", "nado"])

    df['lujo'] = df['titulo'].apply(contar_palabras, palabras=["lujo", "delujo", "deluxe", "delúxe", "lujosa", "lujoso", 
                "lujosas", "lujosos","exclusivo","vip"]) + df['descripcion'].apply(contar_palabras, palabras=["lujo", 
                "delujo","deluxe", "delúxe", "lujosa", "lujoso", "lujosas", "lujosos","exclusivo","vip"])

    df['humilde'] = df['titulo'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
                "económico", "económica", "barata", "barato", "accesible", "baratillo",
                "baratilla", "rebajado", "ganga", "asequible", "módico", "módica","credito","crédito","oferta","oferton",
                "imperdible"]) + df['descripcion'].apply(contar_palabras, palabras=["humilde", "economico", "economica", 
                "económico", "económica", "barata", "barato", "accesible", "baratillo",
                "baratilla", "rebajado", "ganga", "asequible", "módico", "módica",
                "credito","crédito","oferta","oferton","imperdible"]) 

    df['ventana'] = df['titulo'].apply(contar_palabras, palabras=["ventana", "ventanas", 
                "vista", "ventanal","vistas","cristal"]) + df['descripcion'].apply(contar_palabras, palabras=["ventana",
                "ventanas", "vista", "ventanal","vistas","cristal"])

    #df['nuevo'] = df['titulo'].apply(contar_palabras, palabras=["reciente", "recien", "recién", "nueva", "nuevo", "nuevas", 
    #            "nuevos", "estrenar"]) + df['descripcion'].apply(contar_palabras, palabras=["reciente", "recien", "recién", 
    #            "nueva", "nuevo", "nuevas", "nuevos", "estrenar"])

    df['luz'] = df['titulo'].apply(contar_palabras, palabras=["luz", "luminoso", "luminosa","claridad", "luminiscencia", 
            "luminosidad", "iluminación","iluminacion"]) + df['descripcion'].apply(contar_palabras, palabras=["luz", 
            "luminoso", "luminosa","claridad", "luminiscencia","luminosidad", "iluminación","iluminacion"])

    #df['bueno'] = df['titulo'].apply(contar_palabras, palabras=["bueno", "buena", "buenas", "buenos","excelente", 
    #            "excelentes","increible","espectacular"]) + df['descripcion'].apply(contar_palabras, palabras=["bueno", 
    #            "buena", "buenas", "buenos","excelente", "excelentes","increible","espectacular"])

    df['contable'] = df['titulo'].apply(contar_palabras, palabras=["precio"]) + df['descripcion'].apply(contar_palabras, palabras=["precio"])

    #Nuevos Feat

    df['agente'] = df['descripcion'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"]) \
                 + df['titulo'].apply(contar_palabras, palabras=["inmobiliaria", "asesoria", "asesoría", "lider", "re/max", "remax"])

    df['garante'] = df['descripcion'].apply(contar_palabras, palabras=["garante", "garantía", "fiador", "garantizador", "avalista", "garantia",
                    "defensor", "garantías", "garantes", "codeudor"]) + df['titulo'].apply(contar_palabras, palabras=["garante", 
                    "garantía", "fiador", "garantizador", "avalista", "garantia", "defensor", "garantías", "garantes", "codeudor"])

    df['finanza'] =  df['descripcion'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario"\
                     ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocréditp", "ahorro"]) + \
                     df['titulo'].apply(contar_palabras, palabras=["credito", "crédito", "prestamo", "préstamo", "cuotas", "pagos", "hipotecario" \
                     ,"amortizable", "aptocredito", "aptocrédito", "apto-credito", "apto-crédito", "aptocredito", "aptocrédito", "ahorro"]) 

    df['turismo'] =  df['descripcion'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                       "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])\
                     + df['titulo'].apply(contar_palabras, palabras=["playa", "vacaciones", "descanso", "costa", "arena", "mar", "montaña", "monte",
                                                                       "paisaje", "orilla", "rambla", "turista", "turistas", "cordillera", "sierra"])

    df['longitud_desc'] =  df['descripcion'].apply(lambda x: len(x.split()))

    #Nuevas de V4

    df['longitud_titulo'] = df['titulo'].apply(lambda x: len(x))

    df['cant_!'] = df['titulo'].apply(lambda x: x.count('!')) 

    df['tranquilidad'] = df['descripcion'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                            "paz", "calma", "calmo", "quietud"])\
                     + df['titulo'].apply(contar_palabras, palabras=["armonía", "armonia", "tranquilo", "tranqui", "tranquilidad", \
                                                                            "paz", "calma", "calmo", "quietud"])

    df['reparacion'] = df['descripcion'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                                "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada",
                                                                         "refaccion","refacción","refacciones"])\
                     + df['titulo'].apply(contar_palabras, palabras=["reparación", "reparacion", "reparando", "reparar", \
                                "construcción", "construccion", "construyendo", "construllendo", "mantenimiento", "averiado", "averiada"
                                                                         "refaccion","refacción","refacciones"])

    df['mascotas'] = df['descripcion'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                    "gatos", "gato", "gata", "animal", "animales"])\
                     + df['titulo'].apply(contar_palabras, palabras=["mascotas", "mascota", "perros", "perro", "perra", \
                                                    "gatos", "gato", "gata", "animal", "animales"])

    df['accesibilidad'] = df['descripcion'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                    "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])\
                     + df['titulo'].apply(contar_palabras, palabras=["rampa", "discapacitados", "discapacitado", \
                                                    "discapacitada", "lisiado", "lisiada", "silla de ruedas", "lastimado", "heridos"])

    df['normas'] =  df['descripcion'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                    "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                    "ilegal", "legal", "multa", "infraccion", "infracción"])\
                     + df['titulo'].apply(contar_palabras, palabras=["norma", "normas", "regla", "reglas", \
                                                    "prohibido", "prohibida", "denegado", "denegada", "imposible", \
                                                    "ilegal", "legal", "multa", "infraccion", "infracción"])

    df['beneficios'] = df['descripcion'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                    "agregado", "gratuito", "gratuitamente", \
                                                    "regalo"])\
                     + df['titulo'].apply(contar_palabras, palabras=["gratis", "free", "incluido", "incluye", \
                                                    "agregado", "gratuito", "gratuitamente", \
                                                    "regalo"])

    df['conexion'] = df['descripcion'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                       "señal"])\
                     + df['titulo'].apply(contar_palabras, palabras=["wifi", "wi-fi", "internet", "conexión", "conexion", \
                                                                       "señal"])

    df['servicios_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["servicio", "servicios"])

    df['metros_desc'] = df['descripcion'].apply(contar_palabras, palabras = ["metros","m2"])

    df['acabados'] = df['descripcion'].apply(contar_palabras, palabras = ["acabados","acabado","terminacion","terminación"])

    df['plusvalia'] = df['descripcion'].apply(contar_palabras, palabras = ["plusvalia", "plusvalía"])

    #Nuevos de V5

    df['cocina'] = df['descripcion'].apply(contar_palabras, palabras=['cocina', 'cocinas'])

    df['alberca'] = df['descripcion'].apply(contar_palabras, palabras=['alberca'])

    df['negacion'] = df['descripcion'].apply(contar_palabras, palabras = ["no"])

    df['variospisos'] = df['descripcion'].apply(contar_palabras, palabras = ["escalera","escaleras",
                        "ascensor", "elevador", "escalinata", "gradas", "escalerilla"])

    df['vestidor'] = df['descripcion'].apply(contar_palabras, palabras = ["vestidor"])

    #Nuevos de V7

    df['popularidad'] = df['descripcion'].apply(contar_palabras, palabras=["popular", "famoso", "famosa", "famosos", "populares", \
                                                            "conocido", "conocida", "conocidos"])\
                     + df['titulo'].apply(contar_palabras, palabras=["popular", "famoso", "famosa", "famosos", "populares", \
                                                            "conocido", "conocida", "conocidos"])

    df['aeropuerto_cercano'] = df['descripcion'].apply(contar_palabras, palabras=["aeropuerto", "aeropuertos", \
                                                                                 "aeródromo", "aerodromo", "estacion"\
                                                                                 "estación"])\
                     + df['titulo'].apply(contar_palabras, palabras=["aeropuerto", "aeropuertos", \
                                                                                 "aeródromo", "aerodromo", "estacion"\
                                                                                 "estación"])

    df['traslado'] = df['descripcion'].apply(contar_palabras, palabras=["traslado", "trasladar", "llevamos", \
                                                                       "llevar", "traslada"])\
                     + df['titulo'].apply(contar_palabras, palabras=["traslado", "trasladar", "llevamos", \
                                                                       "llevar", "traslada"])

    #df['extranjero'] = df['descripcion'].apply(contar_palabras, palabras=["extranjero", "extranjera", "extranjeros"])\
     #                + df['titulo'].apply(contar_palabras, palabras=["extranjero", "extranjera", "extranjeros"])

    df['dueño'] = df['descripcion'].apply(contar_palabras, palabras=["dueño", "dueña", "dueños", "propietario", "propietarios"])\
                     + df['titulo'].apply(contar_palabras, palabras=["dueño", "dueña", "dueños", "propietario", "propietarios"])

    df['monoambiente'] = df['descripcion'].apply(contar_palabras, palabras=["monoambiente", "monoambientes"]) \
                         + df['descripcion'].apply(contar_frases, frases=["un ambiente", "unico ambiente", "único ambiente"]) \
                         + df['titulo'].apply(contar_palabras, palabras=["monoambiente", "monoambientes"]) \
                         + df['titulo'].apply(contar_frases, frases=["un ambiente", "unico ambiente", "único ambiente"])

    #La renta fija es un tipo de inversión formada por todos los activos financieros en los que el emisor está 
    #obligado a realizar pagos en una cantidad y en un período de tiempo previamente establecidos
    #Es decir, en la renta fija el emisor garantiza la devolución del capital invertido y una cierta rentabilidad.

    df['renta'] =  df['descripcion'].apply(contar_palabras, palabras=["renta", "devolucion", "devolución", \
                                                                     "devolver", "devolvemos", "devuelto", "depósito", \
                                                                     "deposito", "regreso"]) \
                  + df['descripcion'].apply(contar_frases, frases=["renta fija", "regreso del depósito", "regreso del deposito"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["renta", "devolucion", "devolución", \
                                                                     "devolver", "devolvemos", "devuelto", "depósito", \
                                                                     "deposito", "regreso"]) \
                  + df['titulo'].apply(contar_frases, frases=["renta fija", "regreso del depósito", "regreso del deposito"])

    df['oportunidad'] = df['descripcion'].apply(contar_palabras, palabras=["opotunidad", "chance", "imperdible", \
                                                                          "aprovecha", "aprovechar"]) \
                  + df['descripcion'].apply(contar_frases, frases=["ultima oportunidad", "última oportunidad", \
                                                                  "no te lo pierdas", "no lo pierdas", "no dejes pasar"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["opotunidad", "chance", "imperdible", \
                                                                          "aprovecha", "aprovechar"]) \
                  + df['titulo'].apply(contar_frases, frases=["ultima oportunidad", "última oportunidad", \
                                                                  "no te lo pierdas", "no lo pierdas", "no dejes pasar"]) 

    df['antiguedad_prop'] = df['descripcion'].apply(contar_palabras, palabras=["antiguo", "antigua", "antigüedad", \
                                                                              "antiguedad", "viejo", "vieja", \
                                                                              "clásico", "clasico", "clásica", "clasica"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["antiguo", "antigua", "antigüedad", \
                                                                              "antiguedad", "viejo", "vieja", \
                                                                              "clásico", "clasico", "clásica", "clasica"]) 

    df['niños'] = df['descripcion'].apply(contar_palabras, palabras=["niños", "niñas", "niño", "niña", "bebé", \
                                                                      "bebes", "bebe", "hijo", "hija", "hijos", "hijas"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["niños", "niñas", "niño", "niña", "bebé", \
                                                                      "bebes", "bebe", "hijo", "hija", "hijos", "hijas"]) 

    df['fotos'] = df['descripcion'].apply(contar_palabras, palabras=["foto", "fotos", "imagenes", "imágenes", "imagen", \
                                                                    "ilustrado", "ilustración", "ilustracion", "fotografia", \
                                                                    "fotografía", "fotografías", "fotografias"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["foto", "fotos", "imagenes", "imágenes", "imagen", \
                                                                    "ilustrado", "ilustración", "ilustracion", "fotografia", \
                                                                    "fotografía", "fotografías", "fotografias"]) 

    df['lavadero'] = df['descripcion'].apply(contar_palabras, palabras=["lavadero", "lavadora", "lavarropa", "ropa"]) \
                    + df['descripcion'].apply(contar_frases, frases=["cuarto de lavado"]) \
                    + df['titulo'].apply(contar_frases, frases=["cuarto de lavado"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["lavadero", "lavadora", "lavarropa", "ropa"]) 

    #df['camas_separadas'] =  df['descripcion'].apply(contar_frases, frases=["camas separadas", "camas individuales"]) \
    #              + df['titulo'].apply(contar_frases, frases=["camas separadas", "camas individuales"])

    df['$'] = df['descripcion'].apply(contar_palabras, palabras=["$"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["$"])

    df['trabajo'] = df['descripcion'].apply(contar_palabras, palabras=["oficina", "oficio", "trabajo", "trabajador"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["oficina", "oficio", "trabajo", "trabajador"])

    #df['gas'] = df['descripcion'].apply(contar_palabras, palabras=["gas"]) \
    #              + df['titulo'].apply(contar_palabras, palabras=["gas"])

    df['agua'] = df['descripcion'].apply(contar_palabras, palabras=["cañeria", "cañería", "cañerías", "cañerias", \
                                                                   "agua", "aguas", "tubería", "tuberia", "tuberías", "tuberias"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["cañeria", "cañería", "cañerías", "cañerias", \
                                                                   "agua", "aguas", "tubería", "tuberia", "tuberías", "tuberias"])

    df['amenidad'] = df['descripcion'].apply(contar_palabras, palabras=["ameno", "amenidad", "amenidades"]) \
                  + df['titulo'].apply(contar_palabras, palabras=["ameno", "amenidad", "amenidades"])

    df['broker'] = df['descripcion'].apply(contar_palabras, palabras=["broker", "brokers"]) + \
                   df['titulo'].apply(contar_palabras, palabras=["broker", "brokers"])


    #df['dia_publicacion'] = df['dia_publicacion'].apply(dia_semana)

    df['mes_publicacion'] = df['fecha'].dt.month

    df['metrosdescubiertos'] = abs(df['metrostotales']-df['metroscubiertos'])
    df['relacionmetros'] = df['metrostotales']/df['metroscubiertos']

    return df

In [186]:
train = preparar_df(train)
test = preparar_df(test)
train

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,fotos,lavadero,$,trabajo,agua,amenidad,broker,mes_publicacion,metrosdescubiertos,relacionmetros
0,254099,depto tipo a-402,depto interior de 80 15m2 consta de sala com...,Apartamento,1,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,0,0,0,0,0,0,0,8,0.0,1.000000
1,53461,condominio horizontal en venta,p entre sonora y guerrero atr&aacute;s del h...,Casa en condominio,1,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,0,0,0,0,0,0,0,6,88.0,0.671642
2,247984,casa en venta urbi 3 recamaras tonala,descripcion la mejor ubicacion residencial en...,Casa,1,Tonalá,Jalisco,5.0,3.0,2.0,...,0,0,0,0,0,0,0,10,22.0,1.152778
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,1,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,1,0,0,0,0,0,0,3,4.0,1.063492
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,1,Zapopan,Jalisco,10.0,2.0,1.0,...,0,0,0,0,0,0,0,6,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,1,Zinacantepec,Edo. de México,0.0,2.0,2.0,...,1,0,0,0,0,0,0,2,NaN,NaN
239996,259178,casa en condominio a 10 min del centro de toluca,casa con un jardin amplio un cuarto de servic...,Casa,1,Toluca,Edo. de México,0.0,3.0,3.0,...,0,0,0,0,0,0,0,7,50.0,1.250000
239997,131932,nicolas san juan,departamento con excelente ubicación muy cerc...,Apartamento,1,Benito Juárez,Distrito Federal,20.0,2.0,1.0,...,0,0,0,0,0,0,0,3,0.0,1.000000
239998,146867,casa sola javier rojo gomez,casa sola dividida en cuatro departamentos de...,Casa,1,Iztapalapa,Distrito Federal,20.0,4.0,0.0,...,0,0,0,0,0,0,0,12,98.0,0.582979


In [187]:
    train = train.drop(['descripcion', 'titulo', 'id'], axis = 1)
    test = test.drop(['descripcion', 'titulo', 'id'], axis = 1)
    
    #Adiciono servicios
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    test['servicios'] = test['piscina'] + test['gimnasio'] + test['usosmultiples']
    
    #Dropeo los de bajo scoring
    train = train.drop(['escuelascercanas', 'usosmultiples',"centroscomercialescercanos"], axis = 1)
    test = test.drop(['escuelascercanas', 'usosmultiples',"centroscomercialescercanos"], axis = 1)
    
    train['fecha'] = pd.to_datetime(train['fecha']).dt.year
    test['fecha'] = pd.to_datetime(test['fecha']).dt.year
    
    #"Normalizo" la antiguedad
    #train['antiguedad'] = minmax(train['antiguedad'])
    
    #Elimino los residuos
    #train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                       # 'centroscomercialescercanos'], axis = 1)
    
   
      
    df1 = train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']

    PMM = df1.groupby('idzona').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMean"})
    
    test = test.merge(PMM, on='idzona',how='left')
    train = train.merge(PMM, on='idzona',how='left')

    df1 = train.dropna(subset=['metroscubiertos'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMean"})

    
    test = test.merge(PMM, on='idzona',how='left')
    train = train.merge(PMM, on='idzona',how='left')
    
    
    df1 = train.dropna(subset=['metrosdescubiertos'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('idzona').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMean"})

   
    test = test.merge(PMM, on='idzona',how='left')
    train = train.merge(PMM, on='idzona',how='left') 

    df1 = train.dropna(subset=['metrostotales'])
    df1['preciometro']=df1['precio']/df1['metrostotales']
    
    PMM = df1.groupby('ciudad').agg({'preciometro': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometro": "precioMetroMeanC"})

   
    test = test.merge(PMM, on='ciudad',how='left')
    train = train.merge(PMM, on='ciudad',how='left')
    
    df1 = train.dropna(subset=['metroscubiertos'])
    df1['preciometroCub']=df1['precio']/df1['metroscubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroCub': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroCub": "precioMetroCubMeanC"})

   
    test = test.merge(PMM, on='ciudad',how='left')
    train = train.merge(PMM, on='ciudad',how='left')
    
    df1 = train.dropna(subset=['metrosdescubiertos'])
    df1['preciometroDes']=df1['precio']/df1['metrosdescubiertos']

    PMM = df1.groupby('ciudad').agg({'preciometroDes': 'mean'})
    PMM = PMM.reset_index()
    PMM = PMM.rename(columns={"preciometroDes": "precioMetroDesMeanC"})

    train = train.merge(PMM, on='ciudad',how='left')
    test = test.merge(PMM, on='ciudad',how='left')
    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(train, "provincia", "precio") 
    x_train_mean = mean_target_encoding(x_train_mean, "tipodepropiedad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "idzona", "precio")

    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    test = mean_target_decoding(test, "provincia", x_train_mean)
    test = mean_target_decoding(test, "tipodepropiedad", x_train_mean)
    test = mean_target_decoding(test, "ciudad", x_train_mean)
    test = mean_target_decoding(test, "idzona", x_train_mean)
    
    backup = x_train_mean
    
    y_train= x_train_mean['precio']
    x_train = x_train_mean.drop(["precio","provincia","tipodepropiedad","ciudad"], axis=1)
    test.drop(["provincia","tipodepropiedad", "ciudad"], axis=1, inplace = True)
    
    #Voy a minmaxmizar los metros (todos)
    x_train['metroscubiertos'] = minmax(x_train['metroscubiertos'])
    x_train['metrostotales'] = minmax(x_train['metrostotales'])
    x_train['metrosdescubiertos'] = minmax(x_train['metrosdescubiertos'])
    
    test['metroscubiertos'] = minmax(x_test['metroscubiertos'])
    test['metrostotales'] = minmax(x_test['metrostotales'])
    test['metrosdescubiertos'] = minmax(x_test['metrosdescubiertos'])
    


/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/belu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [188]:
x = x_train.values
y = y_train.values
x_train

,direccion,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,...,precioMetroMean,precioMetroCubMean,precioMetroDesMean,precioMetroMeanC,precioMetroCubMeanC,precioMetroDesMeanC,provincia_mean,tipodepropiedad_mean,ciudad_mean,idzona_mean
0,1,NaN,2.0,1.0,2.0,0.153302,0.153302,23533.0,NaN,NaN,...,28035.465800,28500.216385,inf,28581.902992,27296.034458,inf,1.000000,0.502000,0.252228,0.223699
1,1,10.0,3.0,2.0,2.0,0.596698,0.389151,24514.0,19.310205,-99.227655,...,19664.385765,20823.948429,inf,22928.145532,20388.200851,inf,1.000000,0.531972,0.352409,0.255954
2,1,5.0,3.0,2.0,2.0,0.304245,0.356132,48551.0,NaN,NaN,...,9094.704271,10430.830132,inf,7857.412643,8630.123454,inf,0.591751,0.420924,0.047335,0.059961
3,1,1.0,2.0,1.0,1.0,0.113208,0.122642,53666.0,19.301890,-99.688015,...,10682.861852,10557.174343,1.093090e+05,8687.658811,9705.117356,inf,0.737733,0.420924,0.095349,0.101313
4,1,10.0,2.0,1.0,1.0,0.188679,0.188679,47835.0,NaN,NaN,...,14543.135794,13224.113826,inf,15238.389322,14416.321187,inf,0.591751,0.502000,0.200137,0.133593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,1,0.0,2.0,2.0,1.0,0.122642,NaN,53666.0,NaN,NaN,...,10682.861852,10557.174343,1.093090e+05,8687.658811,9705.117356,inf,0.737733,0.420924,0.095349,0.101313
239996,1,0.0,3.0,3.0,3.0,0.436321,0.554245,51954.0,19.294665,-99.692916,...,9472.681135,9444.711501,inf,10180.287486,9643.854824,inf,0.737733,0.420924,0.093463,0.115743
239997,1,20.0,2.0,1.0,2.0,0.290094,0.290094,50003995.0,NaN,NaN,...,31125.395210,28972.193461,inf,28581.902992,27296.034458,inf,1.000000,0.502000,0.252228,0.310143
239998,1,20.0,4.0,0.0,4.0,0.518868,0.287736,24162.0,19.366651,-99.082246,...,23111.591044,19459.079781,3.951779e+04,14421.553231,12816.911225,inf,1.000000,0.420924,0.113976,0.326403


In [189]:
test

,direccion,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,...,precioMetroMean,precioMetroCubMean,precioMetroDesMean,precioMetroMeanC,precioMetroCubMeanC,precioMetroDesMeanC,provincia_mean,tipodepropiedad_mean,ciudad_mean,idzona_mean
0,1,29.0,3.0,NaN,4.0,0.424528,0.117925,NaN,19.408668,-99.246767,...,NaN,NaN,NaN,31729.768869,31591.028842,inf,1.000000,0.420924,0.343439,NaN
1,1,NaN,1.0,1.0,1.0,0.191038,0.191038,113851.0,21.032480,-89.592424,...,11282.289412,10873.132349,inf,7989.137630,9641.221460,inf,0.469103,0.502000,0.141001,0.194760
2,1,0.0,2.0,1.0,2.0,0.478774,NaN,23620.0,19.332829,-99.152913,...,23569.752299,22387.864374,inf,22885.185902,19838.161099,inf,1.000000,0.502000,0.254061,0.198269
3,1,2.0,2.0,2.0,2.0,0.207547,0.577830,129347.0,16.860487,-99.878383,...,18168.001403,16967.442445,inf,14776.086708,15299.990659,inf,0.658701,0.502000,0.186695,0.094803
4,1,10.0,2.0,1.0,1.0,0.872642,NaN,57125.0,19.640482,-99.127273,...,15391.425595,10130.495154,9.057711e+04,8880.658911,8756.561508,inf,0.737733,0.420924,0.049609,0.051682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,1,20.0,4.0,3.0,3.0,0.375000,0.672170,275741.0,19.434821,-99.092517,...,15299.699389,15924.670740,inf,16344.767926,15195.627586,inf,1.000000,0.420924,0.134100,0.145155
59996,1,10.0,3.0,1.0,2.0,NaN,0.813679,57474.0,NaN,NaN,...,10997.366922,8866.459600,inf,9896.181954,8651.693922,inf,0.737733,0.420924,0.064493,0.021581
59997,1,5.0,3.0,2.0,2.0,0.459906,0.372642,72224.0,NaN,NaN,...,7021.866315,6643.697006,inf,9027.296951,8173.222189,inf,0.721200,0.420924,0.095036,0.038836
59998,1,0.0,2.0,1.0,2.0,0.438679,0.294811,83960.0,20.591773,-100.327615,...,11316.758996,10521.197791,inf,10930.021439,11845.780578,inf,0.509073,0.420924,0.160934,0.099600


In [102]:
model_P = lightgbm.LGBMRegressor(learning_rate= 0.05,min_child_samples=180,min_child_weight = 1e-05,
                                num_leaves=35, reg_alpha=10,reg_lambda=1000,subsample=0.9,max_depth=-1,
                                max_bin=800,metric = 'mae',n_estimators= 20000,
                                random_state=314)
model_P.fit(x,y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_bin=800,
              max_depth=-1, metric='mae', min_child_samples=180,
              min_child_weight=1e-05, min_split_gain=0.0, n_estimators=20000,
              n_jobs=-1, num_leaves=35, objective=None, random_state=314,
              reg_alpha=10, reg_lambda=1000, silent=True, subsample=0.9,
              subsample_for_bin=200000, subsample_freq=0)

In [103]:
xgb_model_P = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, eval_metric='mae',
             gamma=0.015, importance_type='gain', learning_rate=0.025,
             max_delta_step=0, max_depth=11, min_child_weight=1, missing=None,
             n_estimators=3500, n_jobs=None, nthread=-1, objective='reg:linear',
             random_state=0, reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.733, verbosity=1)


#Fitteo
xgb_model_P.fit(x_train, y_train)

/home/belu/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/belu/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[21:18:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, eval_metric='mae',
             gamma=0.015, importance_type='gain', learning_rate=0.025,
             max_delta_step=0, max_depth=11, min_child_weight=1, missing=None,
             n_estimators=3500, n_jobs=None, nthread=-1, objective='reg:linear',
             random_state=0, reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.733, verbosity=1)

In [104]:
lgbm_pred = model_P.predict(test)
xgb_pred = xgb_model_P.predict(test)

[4690192.59234725 2032237.1853033  2117391.89744128 ...  704867.33624246
 1189378.59371296 1045492.42569864]


[4866318.5 2059107.4 2253309.8 ...  775709.4 1267102.5 1187180.1]


In [227]:
prediccion = (xgb_pred*1 + lgbm_pred*0)

In [228]:
test_original = pd.read_csv("./test.csv")
predicciones = test_original.loc[:, ['id','ciudad']]
predicciones.rename(columns = {'ciudad' : 'target'}, inplace = True)
predicciones['target'] = prediccion
predicciones = predicciones.set_index('id')

In [229]:
predicciones.to_csv(r'prediccionEnamble1.csv')